In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install -q keras

In [ ]:
!pip install -q pydrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
project_path = '/content/drive/My Drive/Code Documentation Project/'## we will store our data in this drive

In [ ]:
!pip install simpletransformers

In [ ]:
import re
import string
import numpy as np
import pandas as pd


def clean_text(text):

    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\r", " ", text)


    return text

In [ ]:
#### MY MY MY
#!pip install -q iterative-stratification

from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel

import glob

import os
#from sklearn.model_selection import KFold
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import gc
import keras.backend as K
import numpy as np
import pickle
import pandas as pd



##########
##########
############ Change File Name HERE

#file_name_to_evaluate = 'docsmell_extension_sample_1_junaed.xlsx'

#main_df = pd.read_excel(project_path + 'Documentation Smell (Extension)/Dataset/'+file_name_to_evaluate)

#path = r'C:\DRO\DCL_rawdata_files' # use your path

all_files = glob.glob(project_path+"Documentation Smell (Extension)/Dataset/all labelled sample sets-extension/" + "/*.xlsx")
#all_files = glob.glob(project_path+"Documentation Smell (Extension)/Dataset/saner full dataset/" + "/*.xlsx")


main_df = pd.DataFrame()

for filename in all_files:
    df = pd.read_excel(filename)
    try:
      df['Documentation Text'] = df['Documentation Text'].apply(clean_text)
    except:
      print(filename)
      pass
    main_df = main_df.append(df,ignore_index=True)

del all_files
del df

main_df = main_df.dropna(axis=1, how='any') #deleting (auto-generated) extra columns (containing all NaN values

selected_columns = ['Id',	'Method Prototype',	'Documentation Text',	'Fragmented',	'Tangled',	'Excessive Structured',	'Bloated',	'Lazy']

main_df = main_df[selected_columns]


saner_df = pd.read_excel(project_path + 'Documentation Smell (Extension)/Dataset/labelled_dataset_full_SANER.xlsx')
main_df = main_df.append(saner_df,ignore_index=True)

#main_df = main_df.head(n=50) ### to be commented out


del saner_df

print(len(main_df))

print(main_df)

#main_df = main_df.head(1000) #### TO BE COMMENTED OUT


#main_df = main_df.sample(n = 100) #### TO BE COMMENTED OUT
#main_df = main_df.reset_index(drop=True) #### TO BE COMMENTED OUT





#X_train_all = np.array(text)

#print(X_train_all)
#print(y_train_all)





In [ ]:
#from DataPrep.Clean_Texts import clean_text
import pickle
#dataset=dataset.head(10)

text=main_df['Documentation Text']
text=text.map(lambda x: clean_text(x))


In [ ]:
print(text[:5])

In [ ]:
label=main_df.iloc[:,3:8].values

print(label[:5,:])


In [ ]:
def accuracy_multilabel(y_true,y_pred):
    cnt=0

    N=len(y_pred[0])
    if(N==0):
        return 0

    total=len(y_true)

    for i in range(total):
        cnt_temp=0
        for j in range(N):
            if(y_true[i][j]==y_pred[i][j]):
                cnt_temp+=1
        cnt+=cnt_temp

    cnt/=N
    acc=cnt/total

    return acc



def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)


def exact_match(y_true,y_pred):

    N=len(y_pred[0])
    if(N==0):
        return 0

    total=0
    accurate = 0

    for i in range(len(y_pred)):
        for j in range(N):
            if(y_true[i][j]==y_pred[i][j]):
                accurate+=1
            total+=1

    acc=accurate/total

    return acc

In [ ]:
main_df.head()

In [ ]:
model_type = "xlnet"
model_name = "xlnet-base-cased"

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": project_path + "Models/Transformers/output/"+model_type,
    "best_model_dir": project_path + "Models/Transformers/output/"+model_type+"/best_model",
    "use_early_stopping": False,
    "early_stopping_delta": 0.0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize" : True,
    "early_stopping_patience" : 2,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 512,
    "wandb_project": "Simple Sweep",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "evaluate_during_training_verbose" : True,
    "num_train_epochs": 10
}

train_args["max_seq_length"] = 300
train_args["train_batch_size"] = 16
train_args["gradient_accumulation_steps"] = 2
train_args["evaluate_during_training"] = False
train_args["evaluate_during_training_steps"] = 25
train_args["use_early_stopping"] = False
train_args["learning_rate"] = 5e-5



In [ ]:
ll=[]
ll.append([text[1],label[1]])
print(ll)

In [ ]:


!pip install -q iterative-stratification

from sklearn.model_selection import train_test_split
from simpletransformers.classification import MultiLabelClassificationModel

import glob

import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import gc
import keras.backend as K
import numpy as np

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit


from sklearn.metrics import classification_report,precision_recall_fscore_support
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score, hamming_loss




X_train_all = np.array(text)
y_train_all = np.array(label)

print(X_train_all.shape)
print(y_train_all.shape)

num_cross_validation = 5


mskf = MultilabelStratifiedShuffleSplit(n_splits = num_cross_validation,test_size=0.2, random_state=42)

pred_list=[]
cvscores = []
y_true_all_fold = []
pred_binary_all_fold = []

Fold_Index = 1

for train, val in mskf.split(X_train_all, y_train_all):
    gc.collect()
    K.clear_session()
    print('Fold: ', Fold_Index)
    Fold_Index = Fold_Index + 1

    X_train = X_train_all[train]
    X_val = X_train_all[val]
    y_train = y_train_all[train]
    y_val = y_train_all[val]

    ll=[]
    for i in range(len(X_train)):
      ll.append([X_train[i],y_train[i]])

    train_val_df = pd.DataFrame(ll)
    train_val_df.columns = ["text", "labels"]

    ll=[]
    for i in range(len(X_val)):
      ll.append([X_val[i],y_val[i]])

    test_df = pd.DataFrame(ll)
    test_df.columns = ["text", "labels"]



    train_df, val_df = train_test_split(train_val_df, test_size=0.1)


    # Create a ClassificationModel
    model = MultiLabelClassificationModel(model_type, model_name, num_labels=5, args=train_args)

    # Train the model
    model.train_model(train_df, eval_df=val_df)

    pred, raw_outputs = model.predict(test_df['text'].to_list())

    pred_binary=np.array(pred)
    for i in range(len(pred_binary)):
      for j in range(len(pred_binary[i])):
        pred_binary[i][j]=int(1*(pred_binary[i][j]>0.5))
      y_true_all_fold.append(y_val[i])
      pred_binary_all_fold.append(pred_binary[i])

    #report=classification_report(y_true_all_fold,pred_binary_all_fold)



y_true_np = np.array(y_true_all_fold)
y_pred_np = np.array(pred_binary_all_fold)

#print(y_true_np)

categories_list=['Fragmented','Tangled','Excessive Structured','Bloated','Lazy']

# for i in range(len(categories_list)):
#   print('Class: ' + categories_list[i])
#   new_y_true = y_true_np[:,i]
#   new_y_pred = y_pred_np[:,i]
#   print(new_y_pred)
#   print('Accuracy for class ' + categories_list[id] + ': ' + str(accuracy_score(new_y_true,new_y_pred)))
#   print('Classification Report for class ' + categories_list[i] + ': ' + str(classification_report(new_y_true,new_y_pred)))

num_classes = 5
for i in range(num_classes):
  print('Class: ' + str(i))
  new_y_true = y_true_np[:,i]
  new_y_pred = y_pred_np[:,i]

  print('Accuracy for class ' + str(i) + ': ' + str(accuracy_score(new_y_true,new_y_pred)))
  print('Classification Report for class' + str(i) + ': ' + str(classification_report(new_y_true,new_y_pred)))


In [ ]:
from sklearn.metrics import classification_report,precision_recall_fscore_support
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.metrics import accuracy_score, hamming_loss

report=classification_report(y_true_np,y_pred_np)
#report=precision_recall_fscore_support(y_test,pred_binary,average='micro')
print('Classification Report: '+str(report))


precision=precision_score(y_true_np,y_pred_np,average='weighted')
print('Weighted Precision: '+str(precision))

recall=recall_score(y_true_np,y_pred_np,average='weighted')
print('Weighted Recall: '+str(recall))

f1=f1_score(y_true_np,y_pred_np,average='weighted')
print('Weighted F1 Score: '+str(f1))



acc_hard=accuracy_score(y_true_np,y_pred_np)
print('Hard Accuracy: '+str(acc_hard))

#from Custom_Metrics import accuracy_multilabel

acc_custom=accuracy_multilabel(y_true_np,y_pred_np)
print('Custom Accuracy: '+str(acc_custom))

acc_hamming=hamming_score(y_true_np,y_pred_np)
print('Hamming Score: '+str(acc_hamming))

loss_hamming=hamming_loss(y_true_np,y_pred_np)
print('Hamming Loss: '+str(loss_hamming))

acc_exact=exact_match(y_true_np,y_pred_np)
print('Exact Match: '+str(acc_exact))